In [1]:
# turnquest_pygame.py
"""
TurnQuest - Stylized Pygame Turn-Based Battle (single file)

Features:
- Start screen with hero name input (GUI)
- Stylized panels, simple enemy/player "sprites", smooth button hover
- Moves: Slash, Power Strike, First Aid (heal), Flame Shot (burn)
- Items: Potion, Antidote
- Status: Burn (damage over time)
- Move cooldowns, simple enemy AI, XP & leveling
- Loopable encounters, post-battle GUI (Continue / Rest / Quit)
- No console input/output; quit anytime
"""

import pygame
import sys
import random
from dataclasses import dataclass, field
from typing import List, Optional, Callable, Dict

# -------------------- Config --------------------
WIDTH, HEIGHT = 1000, 640
FPS = 60
FONT_NAME = None
BG_COLOR = (18, 20, 28)
PANEL = (28, 34, 45)
ACCENT = (99, 155, 255)
TEXT = (230, 230, 240)
HP_GREEN = (94, 221, 121)
HP_RED = (235, 92, 92)
LOG_BG = (14, 16, 20)
BUTTON = (48, 58, 75)
BUTTON_HOVER = (70, 95, 140)
DISABLED = (70, 70, 80)
FADE_COLOR = (0, 0, 0, 160)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("TurnQuest")
clock = pygame.time.Clock()
font = pygame.font.Font(FONT_NAME, 18)
bigfont = pygame.font.Font(FONT_NAME, 32)
title_font = pygame.font.Font(FONT_NAME, 46)

# -------------------- Data Models --------------------
@dataclass
class Status:
    name: str
    duration: int
    potency: int = 0

@dataclass
class Move:
    name: str
    desc: str
    power: int = 0
    accuracy: float = 1.0
    cooldown: int = 0
    current_cd: int = 0
    target_self: bool = False
    apply_status: Optional[Callable[['Character','Character'], Optional[Status]]] = None

    def ready(self):
        return self.current_cd <= 0

    def trigger_cd(self):
        if self.cooldown > 0:
            self.current_cd = self.cooldown

    def tick_cd(self):
        if self.current_cd > 0:
            self.current_cd -= 1

@dataclass
class Item:
    name: str
    desc: str
    use: Callable[['Character'], str]
    qty: int = 1

@dataclass
class Character:
    name: str
    level: int
    max_hp: int
    hp: int
    atk: int
    defense: int
    speed: int
    moves: List[Move]
    statuses: List[Status] = field(default_factory=list)
    buffs: Dict[str, Status] = field(default_factory=dict)
    xp: int = 0

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, amt: int):
        amt = max(0, amt)
        self.hp = max(0, min(self.max_hp, self.hp - amt))
        return amt

    def heal(self, amt: int):
        if amt <= 0:
            return 0
        healed = max(0, min(self.max_hp - self.hp, amt))
        self.hp += healed
        return healed

    def add_status(self, s: Status):
        for ex in self.statuses:
            if ex.name == s.name:
                ex.duration = s.duration
                ex.potency = s.potency
                return
        self.statuses.append(s)

    def has_status(self, name: str):
        return any(s.name == name for s in self.statuses)

    def tick_statuses(self):
        """Apply start-of-turn statuses. Return list of messages."""
        msgs = []
        for s in self.statuses[:]:
            if s.name == 'Burn':
                dmg = s.potency
                self.take_damage(dmg)
                msgs.append(f"{self.name} takes {dmg} burn damage.")
            s.duration -= 1
            if s.duration <= 0:
                self.statuses.remove(s)
                msgs.append(f"{self.name} is no longer {s.name.lower()}.")
        # tick buffs too
        for k, b in list(self.buffs.items()):
            b.duration -= 1
            if b.duration <= 0:
                del self.buffs[k]
                msgs.append(f"{self.name}'s {k} buff wore off.")
        return msgs

    def get_stat(self, stat: str):
        base = getattr(self, stat)
        buff = self.buffs.get(stat)
        if buff:
            return base + buff.potency
        return base

    def add_buff(self, stat: str, amount: int, duration: int):
        self.buffs[stat] = Status(name=f"buff:{stat}", duration=duration, potency=amount)

    def tick_move_cds(self):
        for m in self.moves:
            m.tick_cd()

# -------------------- Game Data --------------------
def make_player_moves():
    return [
        Move("Slash", "A solid physical strike.", power=18, accuracy=0.95, cooldown=0),
        Move("Power Strike", "Heavy hit, lower accuracy.", power=30, accuracy=0.75, cooldown=2),
        Move("First Aid", "Heal yourself for moderate HP.", power=-25, accuracy=1.0, cooldown=3, target_self=True),
        Move("Flame Shot", "Deal fire damage and may burn.", power=15, accuracy=0.9, cooldown=3,
             apply_status=lambda u, t: Status("Burn", duration=3, potency=5))
    ]

def make_enemy_moves(level):
    return [
        Move("Bite", "Enemy bite.", power=12 + level*2, accuracy=0.95),
        Move("Rage", "Powerful blow.", power=20 + level*3, accuracy=0.8, cooldown=2),
        Move("Spit", "May burn target.", power=8 + level, accuracy=0.85,
             apply_status=lambda u, t: Status("Burn", duration=3, potency=4))
    ]

def make_items():
    return {
        'potion': Item("Potion", "Heals 40 HP.", qty=3, use=lambda c: f"{c.name} healed {c.heal(40)} HP."),
        'antidote': Item("Antidote", "Removes Burn.", qty=1, use=lambda c: _use_antidote(c))
    }

def _use_antidote(c: Character):
    removed = False
    for s in c.statuses[:]:
        if s.name == 'Burn':
            c.statuses.remove(s)
            removed = True
    return f"{c.name} had their burn removed." if removed else f"{c.name} had no status to remove."

def calculate_damage(attacker: Character, defender: Character, base_power: int):
    atk = attacker.get_stat('atk')
    defense = max(1, defender.get_stat('defense'))
    raw = (atk * base_power) / defense
    variation = random.uniform(0.85, 1.15)
    dmg = int(max(1, raw * variation))
    return dmg

def attempt_hit(move: Move):
    return random.random() <= move.accuracy

def enemy_choose_move(enemy: Character, player: Character):
    usable = [m for m in enemy.moves if m.ready()]
    if not usable:
        return None
    if enemy.hp < enemy.max_hp * 0.35:
        candidates = sorted(usable, key=lambda mm: mm.power)
        return candidates[0]
    candidates = sorted(usable, key=lambda mm: mm.power, reverse=True)
    if random.random() < 0.2:
        for m in usable:
            if m.apply_status:
                return m
    return candidates[0]

def generate_enemy(player_level: int) -> Character:
    lvl = max(1, player_level + random.choice([-1, 0, 1]))
    base_hp = 60 + lvl * 10
    enemy = Character(
        name=random.choice(['Goblin', 'Wolf', 'Bandit', 'Slime', 'Rogue']),
        level=lvl,
        max_hp=base_hp,
        hp=base_hp,
        atk=8 + lvl * 2,
        defense=4 + lvl,
        speed=5 + lvl,
        moves=make_enemy_moves(lvl)
    )
    return enemy

# -------------------- UI Widgets --------------------
class Button:
    def __init__(self, rect, text, action=None, font=font, enabled=True):
        self.rect = pygame.Rect(rect)
        self.text = text
        self.action = action
        self.font = font
        self.enabled = enabled
        self.hover = False

    def draw(self, surf):
        col = BUTTON
        if not self.enabled:
            col = DISABLED
        elif self.hover:
            col = BUTTON_HOVER
        pygame.draw.rect(surf, col, self.rect, border_radius=8)
        txt = self.font.render(self.text, True, TEXT)
        txt_r = txt.get_rect(center=self.rect.center)
        surf.blit(txt, txt_r)

    def handle_event(self, ev):
        if ev.type == pygame.MOUSEMOTION:
            self.hover = self.rect.collidepoint(ev.pos)
        elif ev.type == pygame.MOUSEBUTTONDOWN and ev.button == 1:
            if self.enabled and self.rect.collidepoint(ev.pos):
                if self.action:
                    self.action()

# -------------------- Game Class --------------------
class TurnQuestGame:
    def __init__(self):
        # Progress / player
        self.state = 'start'  # start, battle, post_battle, game_over
        self.player_name = "Hero"
        self.player = None
        self.enemy = None
        self.inventory = None
        self.log: List[str] = []
        self.encounters = 0
        self.selected_move = None
        self.message_timer = 0.0  # seconds used to pause between actions
        self.anim_t = 0.0
        # UI
        self.ui_buttons: List[Button] = []
        self.move_buttons: List[Button] = []
        self.item_buttons: List[Button] = []
        self.quit_button = None
        self.continue_button = None
        self.rest_button = None
        self.setup_start_screen()

    # ----- Start screen with name input -----
    def setup_start_screen(self):
        # Input box variables
        self.input_active = False
        self.input_text = ""
        self.name_box = pygame.Rect(WIDTH//2 - 200, HEIGHT//2 - 30, 400, 48)
        # Start and quit buttons
        self.start_btn = Button((WIDTH//2-110, HEIGHT//2+40, 100, 44), "Start", action=self.start_game)
        self.exit_btn = Button((WIDTH//2+10, HEIGHT//2+40, 100, 44), "Quit", action=self.quit_game)

    def start_game(self):
        name = self.input_text.strip() or "Hero"
        self.player_name = name
        self.player = Character(
            name=self.player_name, level=1,
            max_hp=100, hp=100, atk=12, defense=8, speed=10,
            moves=make_player_moves()
        )
        self.inventory = make_items()
        self.encounters = 0
        self.log = []
        self.next_encounter()
        self.state = 'battle'

    def quit_game(self):
        pygame.quit()
        sys.exit(0)

    # ----- Encounter flow -----
    def next_encounter(self):
        self.encounters += 1
        self.enemy = generate_enemy(self.player.level)
        self.push_log(f"A wild {self.enemy.name} (Lv {self.enemy.level}) appears!")
        self.state = 'battle'
        self.create_battle_ui()
        # start-of-battle ticks
        self.tick_start_of_round()

    def create_battle_ui(self):
        self.ui_buttons.clear()
        self.move_buttons.clear()
        self.item_buttons.clear()
        # Move buttons
        left = 28
        base_y = HEIGHT - 170
        btn_w = 220
        btn_h = 46
        for i, mv in enumerate(self.player.moves):
            rect = (left, base_y + i * (btn_h + 8), btn_w, btn_h)
            b = Button(rect, f"{mv.name} {'(CD:'+str(mv.current_cd)+')' if mv.current_cd>0 else ''}", action=lambda m=mv: self.on_move_click(m))
            self.move_buttons.append(b)
            self.ui_buttons.append(b)
        # Items
        x2 = left + btn_w + 18
        for j, key in enumerate(self.inventory.keys()):
            it = self.inventory[key]
            rect = (x2, base_y + j * (btn_h + 8), 180, btn_h)
            b = Button(rect, f"{it.name} x{it.qty}", action=lambda k=key: self.on_item_click(k))
            self.item_buttons.append(b)
            self.ui_buttons.append(b)
        # Pass button
        pass_rect = (x2 + 200, base_y, 120, btn_h)
        self.ui_buttons.append(Button(pass_rect, "Pass", action=self.player_pass))
        # Quit in-game
        self.quit_button = Button((WIDTH-120, 18, 100, 34), "Quit", action=self.quit_game)
        self.ui_buttons.append(self.quit_button)
        self.refresh_button_states()

    def refresh_button_states(self):
        # Update texts and enabled flags
        for b, m in zip(self.move_buttons, self.player.moves):
            cd = f"(CD:{m.current_cd})" if m.current_cd>0 else ""
            b.text = f"{m.name} {cd}"
            b.enabled = m.ready() and self.player.is_alive() and self.enemy.is_alive() and self.state == 'battle'
        for i, key in enumerate(self.inventory.keys()):
            it = self.inventory[key]
            if i < len(self.item_buttons):
                self.item_buttons[i].text = f"{it.name} x{it.qty}"
                self.item_buttons[i].enabled = (it.qty > 0 and self.state == 'battle')
        # Quit always enabled
        if self.quit_button:
            self.quit_button.enabled = True

    def push_log(self, txt: str):
        self.log.insert(0, txt)
        if len(self.log) > 8:
            self.log = self.log[:8]

    # ----- Player interactions -----
    def on_move_click(self, move: Move):
        if not move.ready():
            self.push_log(f"{move.name} is on cooldown ({move.current_cd} turns).")
            return
        if self.message_timer > 0:
            return
        # immediate resolve player's move
        self.resolve_player_move(move)

    def on_item_click(self, key: str):
        if self.message_timer > 0:
            return
        it = self.inventory[key]
        if it.qty <= 0:
            self.push_log("No item left.")
            return
        # use item on player (only)
        res = it.use(self.player)
        it.qty -= 1
        self.push_log(res)
        # tick CDs? we'll end player's action
        self.end_player_action()

    def player_pass(self):
        if self.message_timer > 0:
            return
        self.push_log(f"{self.player.name} passes the turn.")
        self.end_player_action()

    # ----- Move resolution -----
    def resolve_player_move(self, move: Move):
        # Player uses move (instant UI feedback), then enemy acts after short delay
        self.push_log(f"{self.player.name} uses {move.name}!")
        # hit check
        if not attempt_hit(move):
            self.push_log("But it missed!")
            move.trigger_cd()
            self.refresh_button_states()
            self.message_timer = 0.7
            return
        if move.power < 0:  # healing
            amt = -move.power
            healed = self.player.heal(amt)
            self.push_log(f"{self.player.name} heals {healed} HP.")
        else:
            dmg = calculate_damage(self.player, self.enemy, move.power)
            inflicted = self.enemy.take_damage(dmg)
            self.push_log(f"It deals {inflicted} damage to {self.enemy.name} ({self.enemy.hp}/{self.enemy.max_hp}).")
        # status
        if move.apply_status:
            s = move.apply_status(self.player, self.enemy)
            if s:
                self.enemy.add_status(s)
                self.push_log(f"{self.enemy.name} afflicted with {s.name} for {s.duration} turns.")
        move.trigger_cd()
        self.refresh_button_states()
        # check enemy death
        if not self.enemy.is_alive():
            self.push_log(f"{self.enemy.name} was defeated!")
            self.message_timer = 0.6
            # schedule post-battle handling
            self.message_timer_action = self.handle_victory
            return
        # else enemy will act after delay
        self.message_timer = 0.8
        self.message_timer_action = self.enemy_turn

    def enemy_turn(self):
        # Start-of-enemy-turn statuses are applied at round start; here enemy acts
        if not self.enemy.is_alive() or not self.player.is_alive():
            return
        move = enemy_choose_move(self.enemy, self.player)
        if not move:
            self.push_log(f"{self.enemy.name} cannot act.")
            return
        self.push_log(f"{self.enemy.name} uses {move.name}!")
        if not attempt_hit(move):
            self.push_log("It missed!")
            move.trigger_cd()
            self.refresh_button_states()
            self.message_timer = 0.6
            return
        if move.power < 0:
            healed = self.enemy.heal(-move.power)
            self.push_log(f"{self.enemy.name} heals {healed} HP.")
        else:
            dmg = calculate_damage(self.enemy, self.player, move.power)
            inflicted = self.player.take_damage(dmg)
            self.push_log(f"{self.enemy.name} deals {inflicted} damage to {self.player.name} ({self.player.hp}/{self.player.max_hp}).")
        if move.apply_status:
            s = move.apply_status(self.enemy, self.player)
            if s:
                self.player.add_status(s)
                self.push_log(f"{self.player.name} afflicted with {s.name}!")
        move.trigger_cd()
        self.refresh_button_states()
        # check player death
        if not self.player.is_alive():
            self.push_log(f"{self.player.name} was defeated...")
            self.message_timer = 1.2
            self.message_timer_action = self.handle_defeat
            return
        # next round start: tick start-of-round statuses and CDs
        self.message_timer = 0.6
        self.message_timer_action = self.tick_start_of_round

    def end_player_action(self):
        # After player uses item or passes we let the enemy take its turn
        self.message_timer = 0.6
        self.message_timer_action = self.enemy_turn

    # ----- Round / status handling -----
    def tick_start_of_round(self):
        # Called when a new round begins: apply statuses's start-of-turn effects, tick move CDs
        # Player statuses
        pmsgs = self.player.tick_statuses()
        for m in pmsgs:
            self.push_log(m)
        # Enemy statuses
        emsgs = self.enemy.tick_statuses()
        for m in emsgs:
            self.push_log(m)
        # Tick CDs
        self.player.tick_move_cds()
        self.enemy.tick_move_cds()
        self.refresh_button_states()
        # Check if statuses killed someone
        if not self.enemy.is_alive():
            self.push_log(f"{self.enemy.name} was defeated by status!")
            self.message_timer = 0.6
            self.message_timer_action = self.handle_victory
            return
        if not self.player.is_alive():
            self.push_log(f"{self.player.name} was defeated...")
            self.message_timer = 1.2
            self.message_timer_action = self.handle_defeat
            return
        # otherwise it's player's turn (buttons enabled)
        self.message_timer = 0.0
        self.message_timer_action = None

    # ----- Victory / Defeat handlers -----
    def handle_victory(self):
        # award xp, drops, then go to post_battle menu
        xp_gain = self.enemy.level * 25
        self.player.xp += xp_gain
        self.push_log(f"You gain {xp_gain} XP.")
        # random item drops
        if random.random() < 0.4:
            self.inventory['potion'].qty += 1
            self.push_log("You found a Potion!")
        if random.random() < 0.15:
            self.inventory['antidote'].qty += 1
            self.push_log("You found an Antidote!")
        # level up checks
        self.check_level_up()
        self.state = 'post_battle'
        # create post-battle buttons
        self.create_post_battle_ui()

    def handle_defeat(self):
        # Game over: show game over screen that allows restart
        self.state = 'game_over'
        # create game-over UI
        self.create_game_over_ui()

    def check_level_up(self):
        leveled = False
        while self.player.xp >= 100:
            self.player.xp -= 100
            self.player.level += 1
            leveled = True
            hp_gain = random.randint(6, 12)
            atk_gain = random.randint(1, 3)
            def_gain = random.randint(1, 3)
            spd_gain = random.randint(0, 2)
            self.player.max_hp += hp_gain
            self.player.atk += atk_gain
            self.player.defense += def_gain
            self.player.speed += spd_gain
            self.player.hp = self.player.max_hp
            self.push_log(f"*** {self.player.name} leveled up to Lv {self.player.level}! ***")
            self.push_log(f"+{hp_gain} HP, +{atk_gain} ATK, +{def_gain} DEF, +{spd_gain} SPD")
        return leveled

    # ----- Post-battle UI -----
    def create_post_battle_ui(self):
        self.ui_buttons.clear()
        # Continue, Rest, Quit
        mid_x = WIDTH//2 - 160
        y = HEIGHT//2 + 20
        self.continue_button = Button((mid_x, y, 140, 48), "Continue", action=self.post_continue)
        self.rest_button = Button((mid_x + 160, y, 140, 48), "Rest (+30 HP)", action=self.post_rest)
        self.ui_buttons.extend([self.continue_button, self.rest_button])
        self.ui_buttons.append(Button((WIDTH-120, 18, 100, 34), "Quit", action=self.quit_game))

    def post_continue(self):
        self.next_encounter()

    def post_rest(self):
        healed = self.player.heal(30)
        self.push_log(f"You rest and recover {healed} HP.")
        self.next_encounter()

    # ----- Game Over UI -----
    def create_game_over_ui(self):
        self.ui_buttons.clear()
        mid_x = WIDTH//2 - 160
        y = HEIGHT//2 + 20
        self.ui_buttons.append(Button((mid_x, y, 140, 48), "Restart", action=self.restart_game))
        self.ui_buttons.append(Button((mid_x + 160, y, 140, 48), "Quit", action=self.quit_game))

    def restart_game(self):
        # fresh start returns to start screen
        self.state = 'start'
        self.input_text = ""
        self.setup_start_screen()
        self.log = []

    # ----- Drawing helpers -----
    def draw(self, surf):
        surf.fill(BG_COLOR)
        # Top title
        draw_text_center(surf, "TurnQuest", title_font, (WIDTH//2, 36), ACCENT)
        # Left panel: player info
        pygame.draw.rect(surf, PANEL, (18, 80, 460, 160), border_radius=12)
        if self.player:
            draw_text(surf, f"{self.player.name} (Lv {self.player.level})", bigfont, (34, 92))
            draw_text(surf, f"HP: {self.player.hp}/{self.player.max_hp}", font, (34, 130))
            self.draw_bar(surf, 34, 152, 380, self.player.hp, self.player.max_hp, HP_GREEN)
            draw_text(surf, f"ATK: {self.player.get_stat('atk')}  DEF: {self.player.get_stat('defense')}  SPD: {self.player.get_stat('speed')}", font, (34, 176))
            draw_text(surf, "Statuses: " + (", ".join([f"{s.name}({s.duration})" for s in self.player.statuses]) or "None"), font, (34, 200))
        # Right panel: enemy info
        pygame.draw.rect(surf, PANEL, (WIDTH-478, 80, 460, 160), border_radius=12)
        if self.enemy:
            draw_text(surf, f"{self.enemy.name} (Lv {self.enemy.level})", bigfont, (WIDTH-458, 92))
            draw_text(surf, f"HP: {self.enemy.hp}/{self.enemy.max_hp}", font, (WIDTH-458, 130))
            self.draw_bar(surf, WIDTH-458, 152, 380, self.enemy.hp, self.enemy.max_hp, HP_RED)
            draw_text(surf, "Statuses: " + (", ".join([f"{s.name}({s.duration})" for s in self.enemy.statuses]) or "None"), font, (WIDTH-458, 176))
        # Center: enemy sprite / panel
        center_rect = pygame.Rect(WIDTH//2 - 110, 260, 220, 220)
        pygame.draw.rect(surf, (110, 80, 60), center_rect, border_radius=12)
        if self.enemy:
            draw_text_center(surf, self.enemy.name, font, (center_rect.centerx, center_rect.y - 18), TEXT)
        # Log area
        pygame.draw.rect(surf, LOG_BG, (18, 260, WIDTH-36, 120), border_radius=10)
        for i, line in enumerate(self.log[:6]):
            draw_text(surf, line, font, (34, 272 + i * 18))
        # Buttons or start screen
        if self.state == 'start':
            # draw name input box and buttons
            draw_text_center(surf, "Enter your hero's name", bigfont, (WIDTH//2, HEIGHT//2 - 70), TEXT)
            # name box
            pygame.draw.rect(surf, (255,255,255), self.name_box, border_radius=8)
            inner = pygame.Rect(self.name_box.x+4, self.name_box.y+4, self.name_box.w-8, self.name_box.h-8)
            pygame.draw.rect(surf, (18,20,28), inner, border_radius=6)
            # text
            name_surf = font.render(self.input_text or "Hero", True, TEXT)
            name_rect = name_surf.get_rect(midleft=(inner.x+8, inner.centery))
            surf.blit(name_surf, name_rect)
            # buttons
            self.start_btn.draw(surf)
            self.exit_btn.draw(surf)
            # hint
            draw_text_center(surf, "Click the box and type (Enter to start)", font, (WIDTH//2, HEIGHT//2 + 90), (180,180,200))
        elif self.state == 'battle':
            # Draw UI buttons (moves/items/pass/quit)
            for b in self.ui_buttons:
                b.draw(surf)
            # Also show move buttons separately with better style
            for b in self.move_buttons:
                b.draw(surf)
            for b in self.item_buttons:
                b.draw(surf)
            # small info bar
            draw_text(surf, f"Encounter: {self.encounters}", font, (34, HEIGHT-22))
            draw_text_center(surf, f"XP: {self.player.xp}/100", font, (WIDTH//2, 92), (200,200,220))
        elif self.state == 'post_battle':
            # Dim background
            dim = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            dim.fill((0,0,0,140))
            surf.blit(dim, (0,0))
            # Popup
            pw = 520; ph = 240
            px = WIDTH//2 - pw//2; py = HEIGHT//2 - ph//2
            pygame.draw.rect(surf, PANEL, (px, py, pw, ph), border_radius=12)
            draw_text_center(surf, "Encounter Complete", bigfont, (WIDTH//2, py + 40), ACCENT)
            draw_text_center(surf, "Choose an action", font, (WIDTH//2, py + 90), TEXT)
            # draw buttons
            for b in self.ui_buttons:
                b.draw(surf)
        elif self.state == 'game_over':
            dim = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            dim.fill((0,0,0,180))
            surf.blit(dim, (0,0))
            draw_text_center(surf, "Game Over", title_font, (WIDTH//2, HEIGHT//2 - 60), HP_RED)
            for b in self.ui_buttons:
                b.draw(surf)

    def draw_bar(self, surf, x, y, w, cur, mx, color):
        pygame.draw.rect(surf, (40,40,45), (x, y, w, 18), border_radius=8)
        ratio = max(0.0, min(1.0, cur/mx if mx>0 else 0))
        inner_w = int(w * ratio)
        pygame.draw.rect(surf, color, (x, y, inner_w, 18), border_radius=8)

    # ----- Event handling -----
    def handle_event(self, ev):
        if self.state == 'start':
            # namebox interactions and start/exit buttons
            if ev.type == pygame.MOUSEBUTTONDOWN and ev.button == 1:
                if self.name_box.collidepoint(ev.pos):
                    self.input_active = True
                else:
                    self.input_active = False
                self.start_btn.handle_event(ev)
                self.exit_btn.handle_event(ev)
            elif ev.type == pygame.KEYDOWN and self.input_active:
                if ev.key == pygame.K_BACKSPACE:
                    self.input_text = self.input_text[:-1]
                elif ev.key == pygame.K_RETURN:
                    self.start_btn.action()
                else:
                    # limit name length
                    if len(self.input_text) < 24:
                        char = ev.unicode
                        if char.isprintable():
                            self.input_text += char
        elif self.state == 'battle':
            # pass events to UI buttons
            for b in self.ui_buttons:
                b.handle_event(ev)
            for b in self.move_buttons:
                b.handle_event(ev)
            for b in self.item_buttons:
                b.handle_event(ev)
        elif self.state in ('post_battle', 'game_over'):
            for b in self.ui_buttons:
                b.handle_event(ev)

    def update(self, dt):
        # update timers and hover states
        # Update hover for mouse motion
        mouse_pos = pygame.mouse.get_pos()
        # UI hover handled in Button.handle_event on MOUSEMOTION; also update manually
        if self.state == 'start':
            # update button hover
            self.start_btn.hover = self.start_btn.rect.collidepoint(mouse_pos)
            self.exit_btn.hover = self.exit_btn.rect.collidepoint(mouse_pos)
        else:
            for b in self.ui_buttons:
                b.hover = b.rect.collidepoint(mouse_pos)
            for b in self.move_buttons:
                b.hover = b.rect.collidepoint(mouse_pos)
            for b in self.item_buttons:
                b.hover = b.rect.collidepoint(mouse_pos)

        # message timer countdown
        if hasattr(self, 'message_timer') and self.message_timer > 0:
            self.message_timer -= dt
            if self.message_timer <= 0:
                # trigger scheduled action if present
                act = getattr(self, 'message_timer_action', None)
                self.message_timer_action = None
                if act:
                    act()
        # animations (unused heavy)
        self.anim_t += dt * 1.0

        # ensure button states reflect current game state
        if self.state == 'battle':
            self.refresh_button_states()

    # ----- Main loop runner -----
    def run(self):
        running = True
        while running:
            dt = clock.tick(FPS) / 1000.0
            for ev in pygame.event.get():
                if ev.type == pygame.QUIT:
                    running = False
                else:
                    self.handle_event(ev)
            self.update(dt)
            self.draw(screen)
            pygame.display.flip()
        pygame.quit()
        sys.exit(0)

# -------------------- Drawing utility --------------------
def draw_text(surf, text, fnt, pos, color=TEXT):
    txt = fnt.render(text, True, color)
    surf.blit(txt, pos)

def draw_text_center(surf, text, fnt, pos, color=TEXT):
    txt = fnt.render(text, True, color)
    r = txt.get_rect(center=pos)
    surf.blit(txt, r)

# -------------------- Entry point --------------------
if __name__ == "__main__":
    g = TurnQuestGame()
    g.run()


pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

C:\Users\daniel maged\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
